In [177]:
import re
import string
import nltk
from nltk.cluster.util import cosine_distance
import pandas as pd
import numpy as np
from math import log
import networkx as nx

In [4]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thiag\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
art_anime = "O site oficial da adaptação em anime do mangá de comédia romântica Aharen-san wa Hakarenai divulgou o segundo vídeo promocional do mesmo.\nO vídeo anuncia mais três novos dubladores, apresenta a música tema de abertura do grupo TrySail “Hanarenai Kyori” (Inseparable Distance) e revela a estreia do anime para o dia 1º de abril.\nSinopse\nReina Aharen, uma aluna pequena e bonita com uma voz baixa, é muito ruim em determinar distâncias e espaço pessoal. Às vezes ela está a centímetros de seu rosto e, outras vezes, a quilômetros de distância! O único que tenta entender suas travessuras é Raidou Matsuboshi, que está sentado ao lado dela na classe. Ele tem um rosto ameaçador, mas na realidade, ele é um menino gentil com uma imaginação que às vezes pode correr solta. Aharen-san wa Hakarenai segue o estranho par enquanto sua estranha amizade começa a florescer quando Raidou pega a borracha de Aharen para ela. Ela interpreta mal o gesto dele e agora acredita que eles são os melhores amigos, mostrando como as coisas mais simples podem ser os mais complicados dos desafios para eles.\nAharen-san wa Hakarenai estreará portanto no bloco Animeism nos canais TBS e MBS no dia 1 de abril."

In [6]:
stpwrds = "a, agora, ainda, alguém, algum, alguma, algumas, alguns, ampla, amplas, amplo, amplos, ante, antes, ao, aos, após, aquela, aquelas, aquele, aqueles, aquilo, as, até, através, cada, coisa, coisas, com, como, contra, contudo, da, daquele, daqueles, das, de, dela, delas, dele, deles, depois, dessa, dessas, desse, desses, desta, destas, deste, deste, destes, deve, devem, devendo, dever, deverá, deverão, deveria, deveriam, devia, deviam, disse, disso, disto, dito, diz, dizem, do, dos, e, é, ela, elas, ele, eles, em, enquanto, entre, era, essa, essas, esse, esses, esta, está, estamos, estão, estas, estava, estavam, estávamos, este, estes, estou, eu, fazendo, fazer, feita, feitas, feito, feitos, foi, for, foram, fosse, fossem, grande, grandes, há, isso, isto, já, la, lá, lhe, lhes, lo, mas, me, mesma, mesmas, mesmo, mesmos, meu, meus, minha, minhas, muita, muitas, muito, muitos, na, não, nas, nem, nenhum, nessa, nessas, nesta, nestas, ninguém, no, nos, nós, nossa, nossas, nosso, nossos, num, numa, nunca, o, os, ou, outra, outras, outro, outros, para, pela, pelas, pelo, pelos, pequena, pequenas, pequeno, pequenos, per, perante, pode, pude, podendo, poder, poderia, poderiam, podia, podiam, pois, por, porém, porque, posso, pouca, poucas, pouco, poucos, primeiro, primeiros, própria, próprias, próprio, próprios, quais, qual, quando, quanto, quantos, que, quem, são, se, seja, sejam, sem, sempre, sendo, será, serão, seu, seus, si, sido, só, sob, sobre, sua, suas, talvez, também, tampouco, te, tem, tendo, tenha, ter, teu, teus, ti, tido, tinha, tinham, toda, todas, todavia, todo, todos, tu, tua, tuas, tudo, última, últimas, último, últimos, um, uma, umas, uns, vendo, ver, vez, vindo, vir, vos, vós"
stop_word = [n.strip() for n  in stpwrds.split(',')] 

In [30]:
def simple_clear(text):
    
    ''' 
    simples clear, elimined space and break lines 
    :param text: str text
    return: str re.sub(r'\s+'," ", text.lower())
    '''
    
    return re.sub(r'\s+'," ", text.lower())

def normalize_text(text):
    
    ''' 
    harmonizetion of text,separede text in words, 
    remove space, simbols and stop word (words not contribuinls to meaning) 
    :param text: str text
    return: list text_harmonization: list fo words harmonizeds 
    '''
    
    tokens = nltk.word_tokenize(text.lower(), language="Portuguese")
    text_harmonization = [word for word  in tokens if word not in stop_word and word not in (string.punctuation + "“”") and not word.isdigit()]  
    return text_harmonization

In [22]:
sentency = [sentency for sentency in nltk.sent_tokenize()]
normalize_sent = [ " ".join(normalize_text(sent)) for sent in sentency]

TypeError: sent_tokenize() missing 1 required positional argument: 'text'

In [31]:
def variables(text):
    
    '''  
    create aariables for use in calculate of the lunh, starting by text and number of ranking 
    created variables in function 
    tokens: separed text in sentences, setence_clear: applied cleaning in sentences (remove break line,stop words and others), 
    all_words list of the all words in text (aren't stop words) and rank:  n words, most used in text  
    :param text: str 
    :param rank: int -> number to return of words most commons in text
    :return dict {"tokens": tokens, "sentence_clear": clear_sentency, "all_words": words, "rank": rank_words}'''
    
    tokens= nltk.sent_tokenize(text.lower(), language = "Portuguese")
    clear_sentency = [" ".join(normalize_text(sentency)) for sentency in tokens]
    words = [normalize_text(sentency) for sentency in tokens]
    
    return  {"tokens": tokens, "sentence_clear": clear_sentency, "all_words": words,}

In [32]:
var = variables(art_anime)

In [38]:
def unique_words(text_one, text_two):
    
    '''
    return a list of the words uniques in two texts 
    :param text_one : str -> first text 
    :param text_teo : str -> second text
    >return list list(set(var["all_words"][0] + var["all_words"][1])):
    '''
    return list(set(text_one + text_two))

In [56]:
def num_unique_word(text_one, text_two):
    words  = unique_words(text_one, text_two)
    
    unique_text_one = [len(re.findall(word, " ".join(text_one) )) for word in words]
    unique_text_two = [len(re.findall(word, " ".join(text_two) )) for word in words]
    
    return {"nunique": [unique_text_one, unique_text_two], "labels" : words }
    

In [66]:
def normalize_nunique(vector):
    return [float(value / len(vector)) for value in vector]

In [69]:
num_unique = num_unique_word(var["all_words"][0] , var["all_words"][1])
norm_nunq = [normalize_nunique(sent) for sent in num_unique["nunique"]]

In [117]:
words_nuniques = pd.DataFrame(data = norm_nunq, columns = num_unique["labels"])
words_uniques = pd.DataFrame(data = num_unique["nunique"], columns = num_unique["labels"])

In [83]:
words_uniques

,comédia,wa,trysail,três,grupo,site,adaptação,1º,hakarenai,distance,...,vídeo,hanarenai,oficial,abertura,divulgou,estreia,música,abril,novos,revela
0,1,1,0,0,0,1,1,0,1,0,...,1,0,1,0,1,0,0,0,0,0
1,0,0,1,1,1,0,0,1,0,1,...,1,1,0,1,0,1,1,1,1,1


In [95]:
freq = [words_uniques[col].sum() for col in list(words_uniques.columns)]

In [110]:
def calculate_iDF(data):
    count_in_sent = [data[col].apply(
                    lambda x: 1 if x > 0 else 0).sum() for col in list(data.columns) ]
    
    return  [1.0 + log(data.shape[0])/ sent for sent in count_in_sent if sent > 0 and not sent is None]

In [116]:
IDF = calculate_iDF(words_uniques)

In [123]:
def product_if_IDF(data, idf):
    columns = list(data.columns)
    data_process = data.copy()
    for index in range(len(columns)):
        data_process[columns[index]] = data[columns[index]].apply(lambda x: x * idf[index])
    return data_process

In [125]:
TF_IDF = product_if_IDF(words_nuniques, IDF)

In [153]:
TF_IDF_TRAN = TF_IDF.transpose()

In [179]:
TF_IDF_TRAN = TF_IDF_TRAN.reset_index(drop = True)


In [162]:
(TF_IDF_TRAN[0] * IDF).sum() + (TF_IDF_TRAN[1] * IDF).sum() 

2.933786212659348

In [163]:
((TF_IDF_TRAN[0] * IDF) * (TF_IDF_TRAN[1] * IDF)).sum()

0.008532647236123963

In [180]:
product_doc = (TF_IDF_TRAN[1] * IDF).sum() 

In [181]:
query  = (TF_IDF_TRAN[1].apply(lambda x: x**2).sum())**0.5

In [182]:
doc = (sum([i**2 for  i in IDF]))**0.5

In [183]:
product_doc / (query * doc)

0.7970138871545912

In [178]:
cosine_distance( num_unique["nunique"][0], num_unique["nunique"][1])

0.834855435231046